# Tokenization

This is a simulation of the tokenization process. The goal is to take a string of text and break it down into smaller pieces, or tokens. This is often the first step in natural language processing (NLP) tasks.

### Input sentence

In [1]:
sentence = 'Life is short, eat dessert first'

### Simulating the vocabulary

In [2]:
vocabulary = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
print(vocabulary)

{'Life': 0, 'dessert': 1, 'eat': 2, 'first': 3, 'is': 4, 'short': 5}


### Tokenization Process

In [3]:
import torch

sentence_int = torch.tensor([vocabulary[s] for s in sentence.replace(',', '').split()])
print(sentence_int)

tensor([0, 4, 5, 2, 1, 3])


# Embedding

Create vector embeddings for the tokens. Each token is represented as a 16-dimensional vector. Since we have 6 tokens, we will create a 6x16 matrix. The values in the matrix are randomly generated for the purpose of this simulation.

In [ ]:
torch.manual_seed(123)  # embedding initialization is random, so we set a seed for reproducibility
embed = torch.nn.Embedding(6, 16)  # 6 embeddings, 16 dimensions each
embedded_sentence = embed(sentence_int).detach()    # detach() to avoid tracking gradients for this example (i.e. we are not training the embedding)

print(embedded_sentence)
print(embedded_sentence.shape)

tensor([[ 0.3374, -0.1778, -0.3035, -0.5880,  0.3486,  0.6603, -0.2196, -0.3792,
          0.7671, -1.1925,  0.6984, -1.4097,  0.1794,  1.8951,  0.4954,  0.2692],
        [ 0.5146,  0.9938, -0.2587, -1.0826, -0.0444,  1.6236, -2.3229,  1.0878,
          0.6716,  0.6933, -0.9487, -0.0765, -0.1526,  0.1167,  0.4403, -1.4465],
        [ 0.2553, -0.5496,  1.0042,  0.8272, -0.3948,  0.4892, -0.2168, -1.7472,
         -1.6025, -1.0764,  0.9031, -0.7218, -0.5951, -0.7112,  0.6230, -1.3729],
        [-1.3250,  0.1784, -2.1338,  1.0524, -0.3885, -0.9343, -0.4991, -1.0867,
          0.8805,  1.5542,  0.6266, -0.1755,  0.0983, -0.0935,  0.2662, -0.5850],
        [-0.0770, -1.0205, -0.1690,  0.9178,  1.5810,  1.3010,  1.2753, -0.2010,
          0.4965, -1.5723,  0.9666, -1.1481, -1.1589,  0.3255, -0.6315, -2.8400],
        [ 0.8768,  1.6221, -1.4779,  1.1331, -1.2203,  1.3139,  1.0533,  0.1388,
          2.2473, -0.8036, -0.2808,  0.7697, -0.6596, -0.7979,  0.1838,  0.2293]],
       grad_fn=<Embed

# Q, K, V matrices

Q = Query matrix
K = Key matrix
V = Value matrix

For each token, we calculate the Q, K, V matrices. 

The Q (Query) and K (Key) matrices are used together to compute the attention scores (typically through a dot product), which are then passed through a softmax to get the attention weights. These weights are then used to weight the V (Value) matrix to produce the output of the attention mechanism.

So in short:
- Q and K → compute attention scores

- Scores + softmax → attention weights

- Attention weights × V → attention output

For each token x_i, we calculate the Q, K, V matrices as follows:
- q_i = Q * x_i
- k_i = K * x_i
- v_i = V * x_i

Note that the Q, K matrices are size d_k x 16 and the V matrix is size d_v x 16.

---

## Q, K, V Analogy – Like a Search Engine 

- **Query (Q):** The user's search input (e.g., `"cats"`).
- **Key (K):** The tags or keywords describing each document (e.g., `"cat"`, `"animal"`, `"pet"`).
- **Value (V):** The actual content or data in the documents (e.g., the full article about cats).
- **Attention Mechanism:** The model compares the **query** to all **keys** to determine **how relevant** each document is (attention weights), and then uses those weights to **focus on and combine** the corresponding **values** to generate the output.

## Why Q, K, V called projection matrices?

Because in linear algebra:
> A projection is a transformation from one vector space to another, often used to extract certain features or views of the original data.

Here, Q, K, and V each give a different "view" or role of the same input:

- Q is for asking questions

- K is for providing content descriptions (like metadata)

- V is for carrying actual content

The idea is: "Let’s take the same input and linearly project it into three different functional spaces for computing attention."

---

## Self-Attention Mechanism

**Self-attention** is attending to the same text for context.

`sentence` = "Black cat sat on the brown mat."

`tokens` = ["Black", "cat", "sat", "on", "the", "brown", "mat"]

### Self-attention - let's focus on the word: "black"
1. "black" becomes the Query (q)
It asks: "What words are important to me?"

2. It compares itself to every Key:
    - Similarity(q<sub>black</sub>, K<sub>black</sub>)

    - Similarity(q<sub>black</sub>, K<sub>cat</sub>)

    - Similarity(q<sub>black</sub>, k<sub>sat</sub>)

    - …and so on

> **Note:** The similarity is often computed using a dot product or cosine similarity which gives a single value score for each pair of `q` and `k`. This score indicates how much attention the word "black" should pay to each of the other words in the sentence.

3. Then softmax is applied to turn these scores into attention weights (probabilities).
Let’s say after softmax, the query "black" gives highest weight to following keys:

    - "black" itself → 0.5

    - "cat" → 0.4

    - others → 0.1 or less

4. Use those weights on the Value vectors:
    - 0.5 × v<sub>black</sub>

    - 0.4 × v<sub>cat</sub>

    - ... and so on

- Add them up = the new vector representing "black" after attending to relevant context.

> **Note:** The value vector `v` is the actual content of the words represented in a high-dimensional space.



---

We are calculating dot product between q_i and k_j for all i, j. Hence d_q = d_k = 24. 

Using d_v = 28.

## Set Q, K, V

In [11]:
torch.manual_seed(123)

d = embedded_sentence.shape[1]  # d = embedding dimension
d_q, d_k, d_v = 24, 24, 28

# Parameter is a special kind of tensor which are auto assigned to Module.parameters iterator
# This allows us to pass the parameters to an optimizer so they can be updated during training
# Also it auto-sets the Tensor.requires_grad=True flag so we can use them in training
Q = torch.nn.Parameter(torch.randn(d_q, d)) 
K = torch.nn.Parameter(torch.randn(d_k, d))
V = torch.nn.Parameter(torch.randn(d_v, d))

print()
print(f"Q {Q}")
print(f"Q shape: {Q.shape}")
print(f"K {K}")
print(f"K shape: {K.shape}")
print(f"V {V}")
print(f"V shape: {V.shape}")


Q Parameter containing:
tensor([[ 3.3737e-01, -1.7778e-01, -3.0353e-01, -5.8801e-01,  3.4861e-01,
          6.6034e-01, -2.1964e-01, -3.7917e-01,  7.6711e-01, -1.1925e+00,
          6.9835e-01, -1.4097e+00,  1.7938e-01,  1.8951e+00,  4.9545e-01,
          2.6920e-01],
        [-7.7020e-02, -1.0205e+00, -1.6896e-01,  9.1776e-01,  1.5810e+00,
          1.3010e+00,  1.2753e+00, -2.0095e-01,  4.9647e-01, -1.5723e+00,
          9.6657e-01, -1.1481e+00, -1.1589e+00,  3.2547e-01, -6.3151e-01,
         -2.8400e+00],
        [-1.3250e+00,  1.7843e-01, -2.1338e+00,  1.0524e+00, -3.8848e-01,
         -9.3435e-01, -4.9914e-01, -1.0867e+00,  8.8054e-01,  1.5542e+00,
          6.2662e-01, -1.7549e-01,  9.8284e-02, -9.3507e-02,  2.6621e-01,
         -5.8504e-01],
        [ 8.7684e-01,  1.6221e+00, -1.4779e+00,  1.1331e+00, -1.2203e+00,
          1.3139e+00,  1.0533e+00,  1.3881e-01,  2.2473e+00, -8.0364e-01,
         -2.8084e-01,  7.6968e-01, -6.5956e-01, -7.9793e-01,  1.8383e-01,
          2.2935e-

## Computing unnormalized attention weights (aka scores)

### Sample 

In [ ]:
x_2 = embedded_sentence[1]
print(x_2.shape)
query_2 = Q.matmul(x_2) # query_2 = Q @ x_2  # this is equivalent to the above line
key_2 = K.matmul(x_2)
value_2 = V.matmul(x_2)

print(query_2.shape)
print(key_2.shape)
print(value_2.shape)

torch.Size([16])
torch.Size([24])
torch.Size([24])
torch.Size([28])


### Calculate all attention scores

In [15]:
keys = K.matmul(embedded_sentence.T).T  # keys.T = K @ embedded_sentence.T
values = V.matmul(embedded_sentence.T).T  # values.T = V @ embedded_sentence.T

print(f"Keys shape: {keys.shape}")
print(f"Values shape: {values.shape}")

Keys shape: torch.Size([6, 24])
Values shape: torch.Size([6, 28])
